# Pregunta 4

# Estrategia de marketing basada en simulación

In [46]:
# 5.1) Librerías y carga de datos
import pandas as pd
import json
from joblib import load

# Carga el dataset de features (salida de tu FeatureEngineering)
df = pd.read_csv('../data/processed/modeling_dataset.csv')

# Carga los modelos ya entrenados
model_ltv = load('../models/LTV_180/blending.pkl')
model_cac = load('../models/CAC_source_30/ridge.pkl')

# 5.2) Carga las listas de features guardadas al entrenar
with open('../models/LTV_180/blending_features.json', 'r', encoding='utf-8') as f:
    feats_ltv = json.load(f)

with open('../models/CAC_source_30/ridge_features.json', 'r', encoding='utf-8') as f:
    feats_cac = json.load(f)

# Asegúrate de que todas las columnas existen en tu df
feats_ltv = [c for c in feats_ltv if c in df.columns]
feats_cac = [c for c in feats_cac if c in df.columns]



In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36523 entries, 0 to 36522
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   uid                                   36523 non-null  uint64 
 1   n_compras                             36523 non-null  int64  
 2   revenue_total                         36523 non-null  float64
 3   aov                                   36523 non-null  float64
 4   fecha_primera_compra                  36523 non-null  object 
 5   fecha_ultima_compra                   36523 non-null  object 
 6   dias_entre_compras                    36523 non-null  int64  
 7   is_retained                           36523 non-null  int64  
 8   meses_activo                          36523 non-null  float64
 9   gasto_mensual_promedio                36523 non-null  float64
 10  frecuencia_compras                    36523 non-null  float64
 11  dias_primera_se

In [49]:
feats_cac

['n_compras',
 'revenue_total',
 'aov',
 'dias_entre_compras',
 'is_retained',
 'meses_activo',
 'gasto_mensual_promedio',
 'frecuencia_compras',
 'dias_primera_sesion_a_primera_compra',
 'mes_primera_sesion',
 'dia_semana_primera_sesion',
 'es_fin_de_semana_primera_sesion',
 'estacion_primera_sesion',
 'primer_dispositivo',
 'n_sesiones',
 'cohort_year',
 'cohort_month',
 'duracion_promedio_sesion',
 'n_dispositivos_distintos',
 'dias_activo',
 'LTV_180',
 'trend_slope',
 'has_slope']

In [44]:
# 5.3) Preparar X y predecir
# Si tuvieras NaNs en esas columnas, imputa con la mediana
X_ltv = df[feats_ltv].fillna(0.0)
X_cac = df[feats_cac].fillna(0.0)

df['pred_LTV_180'] = model_ltv.predict(feats_ltv)
df['pred_CAC_30']  = model_cac.predict(feats_cac)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# 5.3) Calcular ROMI por fuente
# Asumo que tu columna de canal/fuente se llama 'source'; cámbiala si difiere.
stats = (
    df
    .groupby('source')
    .agg(
        avg_LTV=('pred_LTV_180','mean'),
        avg_CAC=('pred_CAC_30','mean'),
        n_users=('pred_CAC_30','count')
    )
    .reset_index()
)
stats['ROMI'] = stats['avg_LTV'] / stats['avg_CAC']
stats['budget'] = stats['n_users'] * stats['avg_CAC']  # gasto actual por fuente
stats


In [ ]:
# 5.4) Simulación de escenarios
total_budget = stats['budget'].sum()

# Define la fuente objetivo ("Fuente A") para el +10%
source_A = 'A'   # ← cambia a la fuente que quieras testar

#  – Escenario 1: +10% de presupuesto SOLO en source_A (otros iguales, total aumenta)
stats1 = stats.copy()
stats1.loc[stats1['source']==source_A, 'budget'] *= 1.10

#  – Escenario 2: redistribución proporcional del extra +10% de budget de source_A
extra = stats.loc[stats['source']==source_A, 'budget'].iloc[0] * 0.10
stats2 = stats.copy()
stats2['budget'] += stats2['budget'] / total_budget * extra

# Función para calcular ingresos esperados
def apply_revenue(df_stats):
    df = df_stats.copy()
    df['expected_revenue'] = df['budget'] * df['ROMI']
    total_rev = df['expected_revenue'].sum()
    return df, total_rev

base, rev0 = apply_revenue(stats)
esc1, rev1 = apply_revenue(stats1)
esc2, rev2 = apply_revenue(stats2)

print(f"✔ Presupuesto total original: {total_budget:.2f}\n"
      f"  Ingreso esperado original: {rev0:.2f}\n"
      f"✦ Escenario 1 (+10% en {source_A}): ingreso = {rev1:.2f}\n"
      f"✦ Escenario 2 (redistrib. prop.): ingreso = {rev2:.2f}")


In [ ]:
## 5.5) Recomendación y beneficio esperado

if rev1 > rev2:
    mejor = f"Escenario 1: destinar +10% a {source_A}"
    beneficio = rev1 - rev0
else:
    mejor = "Escenario 2: redistribución proporcional"
    beneficio = rev2 - rev0

print(f"**Recomendación:** {mejor}")
print(f"Beneficio adicional esperado: {beneficio:.2f}")
